In [3]:
#importing libraries
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import joblib
import os

#Load the diabetes dataset
diabetes = load_diabetes()
x = diabetes.data
y = diabetes.target

#split the dataset
x_train, x_test, y_train, y_test = train_test_split(
    x,y, test_size =0.3, random_state =42
)
#train a linear regression model
model = LinearRegression()
model.fit(x_train, y_train)


#Evaluation the model
score = model.score(x_test, y_test)
print(f"MOdel R squared Score: {score: 2f}")

#save the trained model locally
os.makedirs('outputs', exist_ok= True)
joblib.dump(model, 'outputs/model.pkl')


MOdel R squared Score:  0.477290


['outputs/model.pkl']

In [4]:
import pandas as pd
df = pd.DataFrame(data=diabetes.data, columns=diabetes.feature_names)
df['target'] = diabetes.target
print(df.head(500))

          age       sex       bmi        bp        s1        s2        s3  \
0    0.038076  0.050680  0.061696  0.021872 -0.044223 -0.034821 -0.043401   
1   -0.001882 -0.044642 -0.051474 -0.026328 -0.008449 -0.019163  0.074412   
2    0.085299  0.050680  0.044451 -0.005670 -0.045599 -0.034194 -0.032356   
3   -0.089063 -0.044642 -0.011595 -0.036656  0.012191  0.024991 -0.036038   
4    0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596  0.008142   
..        ...       ...       ...       ...       ...       ...       ...   
437  0.041708  0.050680  0.019662  0.059744 -0.005697 -0.002566 -0.028674   
438 -0.005515  0.050680 -0.015906 -0.067642  0.049341  0.079165 -0.028674   
439  0.041708  0.050680 -0.015906  0.017293 -0.037344 -0.013840 -0.024993   
440 -0.045472 -0.044642  0.039062  0.001215  0.016318  0.015283 -0.028674   
441 -0.045472 -0.044642 -0.073030 -0.081413  0.083740  0.027809  0.173816   

           s4        s5        s6  target  
0   -0.002592  0.019907 -0.0176

In [5]:
print(diabetes.data.shape)

(442, 10)


In [10]:
from azureml.core import Workspace, Model

#connect to workspace
ws = Workspace.from_config()

#register the model
model = Model.register(
workspace = ws,
model_path = "outputs/model.pkl",
model_name="diabetes-linear-regression-model",
description= "A linear regression model to predict diabetes disease progression."
)

print(f"Registered Model: {model.name}, Version:{model.version}")

Registering model diabetes-linear-regression-model
Registered Model: diabetes-linear-regression-model, Version:10


In [9]:
# Load the diabetes dataset
diabetes = load_diabetes()

# Create a DataFrame for the features
df_features = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)

# Add the target variable (diabetes progression) to the DataFrame
df_target = pd.DataFrame(diabetes.target, columns=['target'])

# Combine features and target into one DataFrame
df = pd.concat([df_features, df_target], axis=1)

# Save to CSV
df.to_csv('diabetes_dataset.csv', index=False)

print("Diabetes dataset saved as 'diabetes_dataset.csv'")

Diabetes dataset saved as 'diabetes_dataset.csv'


In [12]:
from azureml.core import Workspace

subscription_id= "0ab167ad-f649-4056-9c8e-14d7431715ba"
resource_group= "smartdatacohort25"
workspace_name= "yohanmachinelearning"
region= "eastus"

ws = Workspace.get(
    name=workspace_name,
    subscription_id=subscription_id,
    resource_group = resource_group
)
print(f"workspace {ws.name} loaded successfully.")

workspace yohanmachinelearning loaded successfully.


In [13]:
model_path = 'outputs/model.pkl'

In [14]:
model_name = 'diabetes-linear-regression-model'

In [24]:
from azureml.core import Model
register_model = Model.register(
    model_path = model_path,
    model_name = model_name,
    workspace =ws 
)

Registering model diabetes-linear-regression-model


In [35]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
conda_env = Environment('my-conda-env')
conda_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

In [36]:
from azureml.core.webservice import AciWebservice, webservice
aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1
)

In [39]:
service = Model.deploy(
    workspace=ws,
    name='diabetes-prediction-service2',
    models=[register_model],
    inference_config=inference_config,
    deployment_config=aci_config
)
service.wait_for_deployment(show_output=True)

/tmp/ipykernel_3050/3983759604.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(
Service deployment polling reached non-successful terminal state, current service state: Transitioning
Operation ID: aaae86d7-ab42-4179-b051-26240cc53a8d
More information can be found using '.get_logs()'
Error:
{
  "code": "AuthorizationFailed",
  "statusCode": 403,
  "message": "ACI Service request failed. Reason: The client '8386fe20-c1df-46d7-a2b4-8547841bf49b' with object id '8386fe20-c1

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-04-28 15:28:14+00:00 Creating Container Registry if not exists.
2025-04-28 15:28:14+00:00 Use the existing image.
2025-04-28 15:28:17+00:00 Submitting deployment to compute.
Failed


WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Transitioning
Operation ID: aaae86d7-ab42-4179-b051-26240cc53a8d
More information can be found using '.get_logs()'
Error:
{
  "code": "AuthorizationFailed",
  "statusCode": 403,
  "message": "ACI Service request failed. Reason: The client '8386fe20-c1df-46d7-a2b4-8547841bf49b' with object id '8386fe20-c1df-46d7-a2b4-8547841bf49b' does not have authorization to perform action 'Microsoft.ContainerInstance/containerGroups/write' over scope '/subscriptions/0ab167ad-f649-4056-9c8e-14d7431715ba/resourceGroups/smartdatacohort25/providers/Microsoft.ContainerInstance/containerGroups/diabetes-prediction-service2-Nz4f7aSICUCSiop_dz0yDA' or the scope is invalid. If access was recently granted, please refresh your credentials.."
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful termina

In [34]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment 

env = Environment.get(workspace = ws, name= "AzureML-sklearn-1.0-ubuntu20.04-py38-cpu")
#Define Infrence environment
inference_config = InferenceConfig(
    entry_script= "score.py",
    environment = env
)

#Deployment Configuration 
deployment_config = AciWebservice.deploy_configuration(
    cpu_cores =1,
    memory_gb=1 
)

#Deploy the model 
service = Model.deploy(
    workspace = ws,
    name = "diabetes-prediction-service1",
    models = [model],
    inference_config = inference_config,
    deployment_config = deployment_config,
    overwrite =True
)

service.wait_for_deployment(show_output=True)
print(f"Service state: {service.state}")
print(f"Scoring URI: {service.scoring_uri}")

/tmp/ipykernel_3050/83560948.py:19: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


AttributeError: 'LinearRegression' object has no attribute 'id'

In [49]:
from azureml.core import Workspace, Model
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
import time

# 1. Get workspace
ws = Workspace.from_config()

# 2. Prepare deployment with explicit timeout handling
env = Environment.get(workspace=ws, name="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu")
inference_config = InferenceConfig(
    entry_script="score.py",
    environment=env
)

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    auth_enabled=True
)

# 3. Deploy with manual timeout handling
model = Model(ws, name="diabetes-linear-regression-model")
service = Model.deploy(
    workspace=ws,
    name="diabetes-prediction-service",
    models=[model],
    inference_config=inference_config,
    deployment_config=deployment_config,
    overwrite=True
)

# Custom wait logic with timeout (replaces the broken wait_for_deployment(timeout))
max_wait_minutes = 20
start_time = time.time()

while True:
    state = service.state.lower()
    if state == 'healthy':
        print(f"\nSuccess! Scoring URI: {service.scoring_uri}")
        break
    elif 'fail' in state:
        print(f"\nDeployment failed. State: {service.state}")
        print("Check logs with: service.get_logs()")
        break
    
    if (time.time() - start_time) > (max_wait_minutes * 60):
        print("\nTimeout reached. Deployment may still be in progress.")
        print(f"Current state: {service.state}")
        print("Check logs with: service.get_logs()")
        break
    
    print(".", end="", flush=True)
    time.sleep(10)

/tmp/ipykernel_3058/2386266404.py:25: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


................

KeyboardInterrupt: 

In [56]:
from azureml.core import Workspace
ws = Workspace.from_config()
print(f"Workspace name: {ws.name}")
print(f"Resource group: {ws.resource_group}")
print(f"Subscription ID: {ws.subscription_id}")

Workspace name: yohanmachinelearning
Resource group: smartdatacohort25
Subscription ID: 0ab167ad-f649-4056-9c8e-14d7431715ba


In [62]:
# Run this in your terminal
az ml workspace update \
  --name yohanmachinelearning \
  --resource-group smartdatacohort25 \
  --subscription 0ab167ad-f649-4056-9c8e-14d7431715ba \
  --set identity.type="SystemAssigned"

SyntaxError: invalid decimal literal (1417041991.py, line 5)

In [63]:
az role assignment list \
  --assignee $WORKSPACE_OBJ_ID \
  --resource-group smartdatacohort25 \
  --output table

SyntaxError: invalid syntax (3325934576.py, line 1)

In [66]:
!az ml workspace update \
  --name yohanmachinelearning \
  --resource-group smartdatacohort25 \
  --subscription 0ab167ad-f649-4056-9c8e-14d7431715ba \
  --set identity.type="SystemAssigned"


Preview version of extension is enabled by default for extension installation now. Will be disabled in future release. 
Please run 'az config set extension.dynamic_install_allow_preview=true or false' to config it specifically. 
The command requires the extension ml. Do you want to install it now? The command will continue to run after the extension is installed. (Y/n): ^C
